In [1]:
import requests
import numpy as np


def download_data(url,filename):    #download file as data
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nnet_train.dat'
url02 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nnet_test.dat'
download_data(url01, 'hw4_nnet_train.txt')
download_data(url02, 'hw4_nnet_test.txt')
    
    
def readout(filename):
    with open(filename,'r') as FILE:
        n, Dx, Dy = 0 ,[], []
        
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
#            X.insert(0,1)                #insert a constant as bias or as threshold
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x
        y = np.array(Dy)
#    print(x,n)            
    return x,y,n        



trainx, trainy, trainn = readout('hw4_nnet_train.txt')
testx, testy, testn    = readout('hw4_nnet_test.txt') 

In [2]:
#11 Back propagation

import random
import numpy as np
import time


def initw(n,m):
#    w = [[random.uniform(-r,r) for i in range(m)] for i in range(n+1)]  #vector W 有M個 --- dim of w1: n+1*m matrix
    w = np.random.rand(n+1, m)
    w = (w-0.5)*2*r
    return np.array(w)

def updatew(w,x,delta):          # Gradient descent: w = w - eta*x(l-1)[i]*delta(l)[j]
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w[i,j] = w[i,j]-eta*x[i]*delta[j]
    return w
    
def err(w1,w2,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    yhat = np.tanh(w2.T.dot(x1))
    er = y*yhat<0
    eout = er.sum()/n

    return eout
  

m = [1,6,11,16,21]
t = 1        #run t experiments
T = 50000    #run T iteratios per experiment
eta = 0.1
r = 0.1
d = len(trainx[0])
N = trainn


for M in m:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])
        
        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)
            x1 = np.insert(x1,0,1,axis=0)        #add a constant s.t. dimension becomes M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
        
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

M = 1 r = 0.1 eta = 0.1 Eout = 0.236 time consumed: 3.326747417449951
M = 6 r = 0.1 eta = 0.1 Eout = 0.036 time consumed: 6.789524793624878
M = 11 r = 0.1 eta = 0.1 Eout = 0.04 time consumed: 9.974008083343506
M = 16 r = 0.1 eta = 0.1 Eout = 0.04 time consumed: 13.17426085472107
M = 21 r = 0.1 eta = 0.1 Eout = 0.036 time consumed: 16.44048523902893


In [3]:
#12 Back propagation with diff r

def err(w1,w2,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    yhat = np.tanh(w2.T.dot(x1))
    er = y*yhat<0
    eout = er.sum()/n

    return eout


#m = [1,6,11,16,21]
M = 3
t = 1        #run t experiments
T = 50000    #run T iteratios per experiment
eta = 0.1
R = np.array([0,0.001,0.1,10,1000])
d = len(trainx[0])
N = trainn



for r in R:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])


        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)

            x1 = np.insert(x1,0,1,axis=0)  #dimension of M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
        
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

M = 3 r = 0.0 eta = 0.1 Eout = 0.472 time consumed: 4.812514066696167
M = 3 r = 0.001 eta = 0.1 Eout = 0.036 time consumed: 4.914056301116943
M = 3 r = 0.1 eta = 0.1 Eout = 0.036 time consumed: 5.118441104888916
M = 3 r = 10.0 eta = 0.1 Eout = 0.364 time consumed: 4.919104337692261


C:\Users\paes20705\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: overflow encountered in exp
C:\Users\paes20705\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: overflow encountered in exp


M = 3 r = 1000.0 eta = 0.1 Eout = 0.576 time consumed: 5.03713059425354


In [4]:
#13 Back propagation with diff eta

def err(w1,w2,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    yhat = np.tanh(w2.T.dot(x1))
    er = y*yhat<0
    eout = er.sum()/n

    return eout


#m = [1,6,11,16,21]

t = 3        #run t experiments
T = 50000    #run T iteratios per experiment
M = 3
Eta = [0.001,0.01,0.1,1,10]
r = 0.1
d = len(trainx[0])
N = trainn



for eta in Eta:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])


        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)

            x1 = np.insert(x1,0,1,axis=0)  #dimension of M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)
#            print(err(w1,w2,testx,testy,testn))

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

M = 3 r = 0.1 eta = 0.001 Eout = 0.03466666666666667 time consumed: 14.524237871170044
M = 3 r = 0.1 eta = 0.01 Eout = 0.036 time consumed: 14.449984550476074
M = 3 r = 0.1 eta = 0.1 Eout = 0.03733333333333333 time consumed: 14.513198852539062
M = 3 r = 0.1 eta = 1 Eout = 0.528 time consumed: 15.225797414779663
M = 3 r = 0.1 eta = 10 Eout = 0.49066666666666664 time consumed: 15.163686752319336


In [5]:
#14 Back propagation with DNN

def err(w1,w2,w3,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    x2 = np.tanh(w2.T.dot(x1))
    x2 = np.insert(x2,0,np.full(n,1),axis=0)
    yhat = np.tanh(w3.T.dot(x2))
    er = y*yhat<0
    eout = er.sum()/n
    
    return eout


#m = [1,6,11,16,21]

t = 5        #run t experiments
T = 50000    #run T iteratios per experiment
M = 8
m = 3
eta = 0.01
r = 0.1
d = len(trainx[0])
N = trainn


start = time.time()   
nndim = [d,M,m,1]   #d-M-1
Err = 0


for times in range(t):
    w1 = initw(nndim[0],nndim[1])  
    w2 = initw(nndim[1],nndim[2])
    w3 = initw(nndim[2],nndim[3])

    for iteration in range(T):
        randomn = random.randint(0,N-1)
        x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
        x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
        s1 = w1.T.dot(x0)
        x1 = np.tanh(s1)

        x1 = np.insert(x1,0,1,axis=0)  #dimension of 8+1
        s2 = w2.T.dot(x1)
        x2 = np.tanh(s2)
        
        x2 = np.insert(x2,0,1,axis=0)  #dimension of 3+1
        s3 = w3.T.dot(x2)
        x3 = np.tanh(s3)
        
        delta3 = (-8)*(trainy[randomn]-np.tanh(s3))/(np.exp(s3)+np.exp(-s3))
        delta2 = delta3*w3[1:]*4/(np.exp(s2)+np.exp(-s2))
        delta1 = w2[1:].dot(delta2)*4/(np.exp(s1)+np.exp(-s1))
        
        
        w3 = updatew(w3,x2,delta3)
        w2 = updatew(w2,x1,delta2)
        w1 = updatew(w1,x0,delta1)


    Err = Err+err(w1,w2,w3,testx,testy,testn)
    print('err=',err(w1,w2,w3,testx,testy,testn))
        
Eout = Err/t

end = time.time()
print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

err= 0.048
err= 0.052
err= 0.048
err= 0.048
err= 0.052
M = 8 r = 0.1 eta = 0.01 Eout = 0.0496 time consumed: 63.36822557449341


# RBF Network

In [1]:
import requests
import numpy as np


def download_data(url,filename):    #download file as data
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nbor_train.dat'
url02 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nbor_test.dat'
download_data(url01, 'hw4_nbor_train.txt')
download_data(url02, 'hw4_nbor_test.txt')
    
    
def readout(filename):
    with open(filename,'r') as FILE:
        n, Dx, Dy = 0 ,[], []
        
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
#            X.insert(0,1)                #insert a constant as bias or as threshold
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x
        y = np.array(Dy)
#    print(x,n)            
    return x,y,n        


trainx, trainy, trainn = readout('hw4_nbor_train.txt')
testx, testy, testn    = readout('hw4_nbor_test.txt') 

In [2]:
#15 1 Nearest neighbor

def nearestx(trainx,trainy,x):
    i = []
    for obj in x:
        a = ((obj-trainx)**2).T        
        i.append(np.argmin(sum(a)))
    return i

def err(g,y,n):

    er = y*g<0
    eout = er.sum()/n
    return eout

minx = nearestx(trainx,trainy,testx)

gnbor = [trainy[i] for i in minx]

#16
eout = err(gnbor,testy,testn)

print(eout)

0.344


In [285]:
#17
def nearest_k_trx(trx,x,k):
    a = ((x-trainx)**2).T
    s = sum(a)
    i = np.argpartition(s,k)[0:k]  #np.argsort(s)[0:k] #np.argpartition 大部分比 np.argsort快，前者是 partly，只保證前面k个index，后面的不保证，只需要找到前面的 K 个，计算就停下了。
    return i

def guniform(x,y,data):

    yhat = []
    for obj in data:
        n = []
        index = nearest_k_trx(x,obj,k)
        for i in index:
            n.append(y[i])
        n = np.array(n)
        yhat.append(np.sign(sum(n)/k))
    return yhat
            
def err(g,y,n):
    er = y*g<0
    eout = er.sum()/n
    return eout

#17.
k = 5
Data = 'train'

exec('g = guniform(trainx,trainy,{}x)'.format(Data))
exec('print("Err = ",err(g,{}y,{}n))'.format(Data,Data))

#g = guniform(trainx,trainy,trainx)
#err(g,trainy,trainn)

Err =  0.16


In [286]:
#18.
k = 5
Data = 'test'
exec('g = guniform(trainx,trainy,{}x)'.format(Data))
exec('print("Err = ",err(g,{}y,{}n))'.format(Data,Data))

Err =  0.316


## K-means
$$\nabla E_{in} = -2\Sigma^N_{n=1} [x_n \in S_m](x_n-\mu_m) = -2((\Sigma_{x_n\in S_m}X_x)-\left|S_m\right|\mu_m) = 0$$
$$\mu_m\ =\ average\ of\ x_n\ within\ S_m$$

In [3]:
import requests
import numpy as np


def download_data(url,filename):    #download file as data
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nolabel_train.dat'

download_data(url01, 'hw4_nolabel_train.txt')

    
    
def readout(filename):
    with open(filename,'r') as FILE:
        n, Dx, Dy = 0 ,[], []
        
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
#            X.insert(0,1)                #insert a constant as bias or as threshold
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x
        y = np.array(Dy)
#    print(x,n)            
    return x,y,n        


trainx, trainy, trainn = readout('hw4_nolabel_train.txt')

In [7]:
import random
def initmu(trx,N,k):                       #initialize randum centers mu
    mu = []
    for i in random.sample(range(N),k):
        mu.append(trx[i])        
    return np.array(mu)
def initxk(k):
    for i in range(k):                         #令變數x0...xk
        globals()['x'+str(i)] = []

def distanceof(point,points):
    a = (point-points).T
    s = sum(a**2)
    return s                   #return the closest the classs number
    
def classify(data,mu):
    c = [np.argmin(distanceof(pts,mu)) for pts in data]    #[i for i,x in enumerate([1,2,3,2]) if x==2] # => [1, 3] ##if u have duplicates to find
    for i,obj in enumerate(data):
        globals()['x'+str(c[i])].append(obj)
        
def updatex(x):
    x = np.array(x)
    x = sum(x)/len(x) 
    return x                               #return an updated x = 𝑎𝑣𝑒𝑟𝑎𝑔𝑒 𝑜𝑓 𝑥𝑛 𝑤𝑖𝑡ℎ𝑖𝑛 𝑆𝑚

def err(trx,mu):
    s = 0
    initxk(k)
    classify(trx,mu)
    for j in range(k):
        d = distanceof(mu[j],globals()['x'+str(j)])
        s = s + d.sum()
    return s/len(trx)

k = 2
t = 500
Ein = 0.0
for i in range(t):
    Er = 0
    Err = 1
    
    mu = initmu(trainx,trainn,k)               #initialize mu
    initxk(k)                                  #令變數x0...xk = []
    classify(trainx,mu)                        #classify trainx to x0...xk by mu 
    
    while Err!=Er:
        mu = []
        Er = Err
        for i in range(k):                             #update x0...xk
            mu.append(updatex(locals()['x'+str(i)]))   #update mu by averaging xk
            np.array(mu)                               
        Err = err(trainx, mu)                          #classify xk by updated mu and calculate err
#        print(Err)
    Ein = Ein+Err
    
Ein = Ein/t
print('k = ',k, 'Ein = ',Ein)


k =  2 Ein =  2.388187403180245


In [5]:
a = [1,2,3,4,5,6,7,8,9,41,10,11,21,12,13,45,14,15]
a.append(4)
a
#[i for i in a if i>=10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 41, 10, 11, 21, 12, 13, 45, 14, 15, 4]

In [6]:
a=np.array([[1,1,1,1],[1,2,5,6],[2,3,4,8]])
b=np.array([1,2,5,6])
np.argmin(sum((b-a).T)**2)

1